The input files should be `.mcd`,

First let's import some necessary scripts and setup the output folder

In [ ]:
from pathlib import Path
from imcpipe import mcd2analysis, \
                    panel_csv_checker, \
                    create_spatialtis_folder

output_folder = Path("./IMC-Analysis")

You should have a csv file specify your metal, which channels are used in the analysis and segmentation.

| target | metal | full | seg |
| --- | --- | --- | --- |
| Pan-Keratin | Nd148 | 1 | 0 |
| SMA | Pr141 | 1 | 1 |
| CD45 | Sm152 | 1 | 0 |
| DNA | Ir191 | 1 | 0 |
| DNA | Ir193 | 1 | 1 |

The `analysis_stacks` has two parts:

```analysis_stacks = (('seg','_seg'), ('full', '_full'))```

(The column of segmentation channels, the suffix of segmentation images)

(The column of analysis channels, the suffix of analysis images)

In [ ]:
mcdfiles = ['1.mcd', '2.mcd']

analysis_stacks = (('seg','_seg'), ('full', '_full'))
metal_column = "metal"
panel_csv_file = panel_csv_checker("./meta.csv", metal_column)

In [ ]:
mcd2analysis(mcdfiles, output_folder, panel_csv_file, analysis_stacks, metal_column)

Afterwards, following the bodermillerLab's instruction on segmentation to acquire mask images

You only need to do 3 steps

### Prepare ilastik

Load pipeline file: `1-prepare_ilastik.cppipe`

1. File list: choose all files in the `analysis` subfolder

2. Default Output Folder: Choose the `ilastik` subfolder


### Train a pixel classifier

1. Make a new `pixel classification project`.

2. Add the .h5 random crops: `Raw data` -> `Add Seperate Images` -> Select all .h5 images in the `ilastik` subfolder.

3. Select suitable features (or just everything >= 1 pixels)

4. Label
    1. Nuclei
    2. Cytoplasma/membrane
    3. Background

5. Export prob
    - Export Settings -> Source: Probabilities -> Choose Export Image Settings:
        - Convert to datatype: Unsigned Integer 16 bit
        - Renormalize: check
        - Format: Tiff
        - File: leave default
    - Export all

6. Batch processing
    - Batch processing: -> Select raw data files -> select all `_s2.h5` files in the `tiff` folder. (sort by filetype, select all `h5` files).

### Segment ilastik

1. File list: choose all files in the `analysis` subfolder

2. Default Output Folder: Choose the `ilastik` subfolder

And then we could transform the images into anndata using spatialtis

In [ ]:
create_spatialtis_folder(output_folder)

In [ ]:
import numpy as np
import pandas as pd
import spatialtis as st
import scanpy as sc
import anndata as ad

In [ ]:
var = pd.read_csv(panel_csv_file)[['Antibodies', 'metal']]

reader = st.read_ROIs(entry=(output_folder / "spatialtis"),
                      obs_names=['ROI'],
                      var=var,
                      mask_pattern="mask",
                      img_pattern="full")

data = reader.to_anndata(mp=True)
data

And then we use `scanpy` to annotate cell type

In [ ]:
sc.pp.normalize_total(data, target_sum=100)
sc.pp.log1p(data)
sc.tl.pca(data, svd_solver='arpack')
sc.pl.pca_variance_ratio(data, log=True)
sc.pp.neighbors(data, n_neighbors=10, n_pcs=5)
sc.tl.umap(data)
sc.tl.leiden(data, resolution=0.1)

sc.tl.rank_genes_groups(data, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(data, n_genes=4, sharey=False, gene_symbols="target")

Actually label the cells

In [ ]:
mapper = {
    0: 'Stromal Cell',
    1: 'Unknown',
    2: 'Leukocytes',
    3: 'Epithelial Cell',
}

cell_type = [mapper[int(i)] for i in data.obs.leiden]
data.obs['cell_type'] = cell_type

sc.pl.umap(data, color='cell_type', legend_loc='on data', title='', frameon=False)


Remember to save the file at this moment

In [ ]:
data_file = Path("data.h5ad")
data.write(data_file)

Now let's do some analysis

In [ ]:
import spatialtis.plotting as sp
from spatialtis import CONFIG

CONFIG.EXP_OBS = ["ROI"]
CONFIG.CELL_TYPE_KEY = "cell_type"
CONFIG.MARKER_KEY = "Antibodies"
CONFIG.CENTROID_KEY = "centroid"

To run the neighborhood analysis

In [ ]:
st.find_neighbors(data, expand=8)
st.neighborhood_analysis(data)
sp.neighborhood_analysis(data, use="graph_static")